In [1]:
from itertools import repeat
import collections.abc

In [2]:
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return x
        return tuple(repeat(x, n))
    return parse


In [9]:
to_2tuple = _ntuple(2)

In [3]:
import torch.nn as nn
import torch

In [10]:
m1 = nn.MaxPool2d(3, 2, 1)
m2 = nn.AvgPool2d(3, 2, 1)
input = torch.randn(20, 16, 25, 25)
output1 = m1(input)
output2 = m2(input)

In [11]:
output1.shape, output2.shape

(torch.Size([20, 16, 13, 13]), torch.Size([20, 16, 13, 13]))